In [8]:
from langchain_together import TogetherEmbeddings
from langchain_core.documents import Document
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os

In [9]:
# Set API keys
os.environ["TOGETHER_API_KEY"] = "tgp_v1_Gdl66OKThh1KsJjEym9JEgDMqFWqd6bXtlZhviYqf34"
os.environ["PINECONE_API_KEY"] = "pcsk_3875g1_PSfiVC6hgEBa7mPwUMFf6dbhmZa68JiueGaf5eSYDwKoyt8JABHRYsirkcLfRnm"
os.environ["PINECONE_ENVIRONMENT"] = "gcp-starter"  # Replace with your Pinecone environment

In [14]:
# Initialize Pinecone

pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

print(pc.list_indexes())

{'indexes': [{'dimension': 768,
              'host': 'langchain-embeddings-h7crht6.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain-embeddings',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 1536,
              'host': 'movie-index-h7crht6.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'movie-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [15]:
index_name = "langchain-embeddings"
if index_name not in pc.list_indexes().names():
    pc.create_index(
            name=index_name,
            dimension=768,  # Adjust dimension as per embedding model
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )  # Adjust dimension as per embedding model
    print("Index created:", index_name)
index = pc.Index(index_name)

In [16]:
filepath = r"D:\Python_01\Python_01\Langchain01\Introduction_to_Data_and_Data_Science.pdf"

In [17]:
reader = PdfReader(filepath)
text = ""
for page in reader.pages:
    text += page.extract_text()

In [18]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_text(text)
print(chunks[0])

Analysis vs Analytics 
Alright! So… 
Let’s discuss the not-so-obvious differences 
between the terms analysis and analytics. 
Due to the similarity of the words, some people 
believe they share the same meaning, and thus 
use them interchangeably. Technically, this 
isn’t correct. There is, in fact, a distinct 
difference between the two. And the reason 
for one often being used instead of the other 
is the lack of a transparent understanding 
of both. 
So, let’s clear this up, shall we? 
First, we will start with analysis. 
Consider the following… 
You have a huge dataset containing data of 
various types. Instead of tackling the entire 
dataset and running the risk of becoming overwhelmed, 
you separate it into easier to digest chunks 
and study them individually and examine how 
they relate to other parts. And that’s analysis 
in a nutshell. 
One important thing to remember, however, 
is that you perform analyses on things that 
have already happened in the past. Such as


In [19]:
embedding = TogetherEmbeddings(
    model="BAAI/bge-base-en-v1.5",
    api_key=os.getenv("TOGETHER_API_KEY")
)

In [29]:
# Insert documents (Create) using Pinecone client directly

ids = []
for i, chunk in enumerate(chunks):
    # Get embedding vector for the chunk
    embedding_vector = embedding.embed_documents([chunk])[0]
    # Create a unique ID for each chunk
    doc_id = f"doc-{i}"
    # Prepare metadata
    metadata = {"text":chunk,"source": filepath, "chunk_index": i}
    # Upsert into Pinecone
    index.upsert([
        {
            "id": doc_id,
            "values": embedding_vector,
            "metadata": metadata
        }
    ])
    ids.append(doc_id)
    print(f"Stored chunk {i+1}/{len(chunks)}: {doc_id}")

Stored chunk 1/10: doc-0
Stored chunk 2/10: doc-1
Stored chunk 3/10: doc-2
Stored chunk 4/10: doc-3
Stored chunk 5/10: doc-4
Stored chunk 6/10: doc-5
Stored chunk 7/10: doc-6
Stored chunk 8/10: doc-7
Stored chunk 9/10: doc-8
Stored chunk 10/10: doc-9


In [23]:
# Read (Retrieve) all documents using Pinecone client directly

# Get all vector IDs (you should have stored them in 'ids' during upsert)
# If you don't have the list, you can use index.describe_index_stats() to get all IDs
stats = index.describe_index_stats()
all_ids = []
for ns in stats['namespaces'].values():
    all_ids.extend(ns['vector_count'] * [None])  # Pinecone does not return IDs here, so you should track them during upsert

# If you have the IDs list from upsert:
if ids:
    response = index.fetch(ids)
    vectors = response['vectors']
    if vectors:
        first_id = list(vectors.keys())[0]
        first_vector = vectors[first_id]
        print("First document ID:", first_id)
        print("Metadata:", first_vector.get("metadata"))
        print("Embedding:", first_vector.get("values"))
    else:
        print("No documents found.")
else:
    print("No documents found.")

First document ID: doc-9
Metadata: {'chunk_index': 9.0, 'source': 'D:\\Python_01\\Python_01\\Langchain01\\Introduction_to_Data_and_Data_Science.pdf'}
Embedding: [0.0597715713, -0.004136445, -0.0171964187, 0.0175933652, 0.0578179024, 0.00988055579, 0.0270751454, 0.00988928322, -0.0281789955, -0.0268676318, 0.00739794085, 0.0149527295, -0.0520406477, 0.00874528103, -0.00550212478, 0.0719424337, 0.0503538512, -0.0436136238, 0.0299655143, 0.0155904619, -0.0408670083, 0.00974401645, -0.0435893908, -0.00303584, 0.0253604315, -0.0175811648, -0.0180733278, -0.0201939587, -0.028431125, -0.0474586636, 0.0401969329, 0.0117204497, -0.0195613597, -0.0277155321, -0.00612308225, 0.0220927633, 0.00454561133, 0.0490049794, -0.0363558941, -0.0266745239, -0.0287531167, 0.0260824095, -0.0191170238, -0.0117334686, -0.0450131036, -0.00181754841, -0.0188378189, 0.0323245898, 0.0292579085, -0.0146313719, -0.0416858234, 0.0730824694, 0.030516794, -0.0113945203, 0.00240987446, 0.0469481573, 0.0187151, -0.051929

In [25]:
# Update an existing record in Pinecone DB using the Pinecone client

# Suppose you want to update the record with ID 'doc-0'
update_id = "doc-0"
updated_content = "Updated content for Pinecone."
updated_metadata = {"source": filepath, "chunk_index": 0, "updated": True}

# Get new embedding for the updated content
updated_embedding = embedding.embed_documents([updated_content])[0]

# Upsert the updated vector (same ID, new values/metadata)
index.upsert([
    {
        "id": update_id,
        "values": updated_embedding,
        "metadata": updated_metadata
    }
])
print(f"Document {update_id} updated in Pinecone")

Document doc-0 updated in Pinecone


In [26]:
ids = ["doc-0", "doc-1"]  # Example IDs to fetch
response = index.fetch(ids)
print(response)

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'doc-0': {'id': 'doc-0',
                       'metadata': {'chunk_index': 0.0,
                                    'source': 'D:\\Python_01\\Python_01\\Langchain01\\Introduction_to_Data_and_Data_Science.pdf',
                                    'updated': True},
                       'values': [0.0154512189,
                                  -0.0227949861,
                                  0.00854001939,
                                  0.0258591082,
                                  0.0479440056,
                                  -0.0146586,
                                  0.0302150827,
                                  0.0280567557,
                                  -0.0160668958,
                                  -0.042347461,
                                  -0.0270711463,
                                  -0.0118764536,
                                  -0.0885571837,
                                  0.022869844,


In [30]:
# Similarity search in Pinecone DB

query_text = "programming languages used in data science"  # Your search query
query_embedding = embedding.embed_documents([query_text])[0]

# Query Pinecone for top-k similar vectors
top_k = 3
search_response = index.query(
    vector=query_embedding,
    top_k=top_k,
    include_metadata=True
)

# Print results
for match in search_response['matches']:
    print(f"ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadata: {match.get('metadata')}")
    print("---")

ID: chunk-4
Score: 0.783504307
Metadata: {'text': 'can have their unique meanings too. \nMore of this will be explained in the next \nvideo which aims to simplify these, as well as many more with a fantastic diagram. So, \nlet’s move on! \nProgramming Languages & Software Employed in Data Science - All the Tools You \nNeed \nAlright! So… \nHow are the techniques used in data, business \nintelligence, or predictive analytics applied \nin real life? \nCertainly, with the help of computers. \nYou can basically split the relevant tools \ninto two categories—programming languages \nand software. \nKnowing a programming language enables you \nto devise programs that can execute specific \noperations. Moreover, you can reuse these \nprograms whenever you need to execute the \nsame action. \nAs you can see from the infographic, R, and \nPython are the two most popular tools across \nall columns. Their biggest advantage is that \nthey can manipulate data and are integrated \nwithin multiple dat